In [1]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
from datetime import datetime

def generate_and_save_plots(df):
    """Generate and save plots as images using matplotlib."""
    try:
        # Platform distribution
        plt.figure(figsize=(8, 6))
        df['platform'].value_counts().plot(kind='pie', autopct='%1.1f%%', title="Phân bố sản phẩm theo nền tảng")
        plt.ylabel('')
        plt.savefig('platform_dist.png')
        plt.close()

        # Rating distribution
        plt.figure(figsize=(8, 6))
        df['rating'].plot(kind='hist', bins=20, title="Phân bố điểm đánh giá")
        plt.xlabel('Rating')
        plt.savefig('rating_dist.png')
        plt.close()

        # Price distribution
        plt.figure(figsize=(8, 6))
        df['salePrice'].plot(kind='hist', bins=30, title="Phân phối giá sản phẩm")
        plt.xlabel('Price')
        plt.savefig('price_dist.png')
        plt.close()

        # Price trends
        trends = df.groupby(['platform', 'scraped_timestamp'])['salePrice'].mean().unstack(0)
        plt.figure(figsize=(10, 6))
        trends.plot(title="Xu hướng giá theo thời gian")
        plt.ylabel('Mean Price')
        plt.savefig('price_trends.png')
        plt.close()
        
    except Exception as e:
        print(f"Error generating plots: {e}")
        raise

def generate_html_report(df):
    """Generate a comprehensive HTML report with insights"""
    # Calculate key metrics
    total_products = len(df)
    platform_breakdown = df['platform'].value_counts(normalize=True) * 100
    avg_rating = df['rating'].mean()
    avg_price = df['salePrice'].mean()
    price_range = df['salePrice'].max() - df['salePrice'].min()

    # Create the HTML content with more detailed insights
    html_content = f"""
    <html>
        <body style="font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px;">
            <h1 style="color: #333; border-bottom: 2px solid #0066cc; padding-bottom: 10px;">
                E-commerce Sales Analysis Report
            </h1>
            
            <div style="background-color: #f4f4f4; padding: 15px; border-radius: 5px; margin-bottom: 20px;">
                <h2>Executive Summary</h2>
                <p><strong>Report Generated:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
                <p><strong>Total Products Analyzed:</strong> {total_products:,}</p>
                <p><strong>Average Product Rating:</strong> {avg_rating:.2f}/5</p>
                <p><strong>Average Product Price:</strong> ${avg_price:.2f}</p>
                <p><strong>Price Range:</strong> ${price_range:.2f}</p>
            </div>

            <h2>Detailed Visual Analysis</h2>
            
            <h3>Platform Distribution</h3>
            <p>Breakdown of product distribution across different e-commerce platforms:</p>
            {''.join([f'<p>• {platform}: {percentage:.2f}%</p>' for platform, percentage in platform_breakdown.items()])}
            <img src="cid:platform_dist" alt="Platform Distribution" style="max-width: 100%; height: auto;">
            
            <h3>Rating Distribution</h3>
            <p>Insights into product ratings and customer satisfaction:</p>
            <img src="cid:rating_dist" alt="Rating Distribution" style="max-width: 100%; height: auto;">
            
            <h3>Price Distribution</h3>
            <p>Analysis of product pricing strategies:</p>
            <img src="cid:price_dist" alt="Price Distribution" style="max-width: 100%; height: auto;">
            
            <h3>Price Trends Over Time</h3>
            <p>Tracking average prices across different platforms:</p>
            <img src="cid:price_trends" alt="Price Trends" style="max-width: 100%; height: auto;">
        </body>
    </html>
    """
    return html_content

def send_email(sender, app_password, recipient):
    # Connect to database and get data
    con = sqlite3.connect('e-com.sqlite')
    df = pd.read_sql_query("SELECT * FROM fact_sales;", con)
    con.close()

    # Convert timestamp to datetime
    df['scraped_timestamp'] = pd.to_datetime(df['scraped_timestamp'])

    # Generate and save plots
    generate_and_save_plots(df)

    # Create message container
    msg = MIMEMultipart('related')
    msg['Subject'] = f'E-commerce Sales Analysis Report - {datetime.now().strftime("%Y-%m-%d")}'
    msg['From'] = sender
    msg['To'] = recipient

    # Generate HTML content with insights
    html_content = generate_html_report(df)
    html_part = MIMEText(html_content, 'html')
    msg.attach(html_part)

    # Attach the images inline
    images = {
        'platform_dist.png': 'platform_dist',
        'rating_dist.png': 'rating_dist',
        'price_dist.png': 'price_dist',
        'price_trends.png': 'price_trends'
    }
    
    for image_path, cid in images.items():
        with open(image_path, 'rb') as img_file:
            img = MIMEBase('application', 'octet-stream')
            img.set_payload(img_file.read())
            encoders.encode_base64(img)
            img.add_header('Content-Disposition', f'inline; filename={image_path}')
            img.add_header('Content-ID', f'<{cid}>')
            msg.attach(img)

    try:
        # Create secure SSL/TLS connection
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.login(sender, app_password)
        
        # Send email
        server.sendmail(sender, recipient, msg.as_string())
        server.quit()
        print("Email sent successfully!")
        
    except Exception as e:
        print(f"Error sending email: {str(e)}")

if __name__ == '__main__':
    # Your email configuration
    sender = "~~~~~~~~~~~@gmail.com"
    app_password = "~~~~~~~~~~~~~"
    recipient = "~~~~~~~~~~~~~~@gmail.com"
    
    # Send the report
    send_email(sender, app_password, recipient)

Email sent successfully!


<Figure size 1000x600 with 0 Axes>

In [5]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
from datetime import datetime

def generate_and_save_plots(df):
    """Tạo và lưu các biểu đồ phân tích dữ liệu"""
    try:
        # Phân bố nền tảng
        plt.figure(figsize=(10, 7))
        platform_counts = df['platform'].value_counts()
        platform_counts.plot(kind='pie', autopct='%1.1f%%', 
                              colors=['#ff9999','#66b3ff','#99ff99','#ffcc99'],
                              startangle=90, 
                              title="Phân Bố Sản Phẩm Theo Nền Tảng")
        plt.ylabel('')
        plt.tight_layout()
        plt.savefig('platform_dist.png', dpi=300)
        plt.close()

        # Phân bố điểm đánh giá
        plt.figure(figsize=(10, 7))
        df['rating'].plot(kind='hist', bins=20, 
                          color='#66b3ff', 
                          edgecolor='black',
                          title="Phân Bố Điểm Đánh Giá Sản Phẩm")
        plt.xlabel('Điểm Đánh Giá')
        plt.ylabel('Số Lượng Sản Phẩm')
        plt.tight_layout()
        plt.savefig('rating_dist.png', dpi=300)
        plt.close()

        # Phân phối giá
        plt.figure(figsize=(10, 7))
        df['salePrice'].plot(kind='hist', bins=30, 
                              color='#99ff99', 
                              edgecolor='black',
                              title="Phân Phối Giá Sản Phẩm")
        plt.xlabel('Giá Sản Phẩm (VNĐ)')
        plt.ylabel('Số Lượng Sản Phẩm')
        plt.tight_layout()
        plt.savefig('price_dist.png', dpi=300)
        plt.close()

        # Xu hướng giá theo thời gian
        plt.figure(figsize=(12, 8))
        trends = df.groupby(['platform', 'scraped_timestamp'])['salePrice'].mean().unstack(0)
        trends.plot(marker='o', linewidth=2, 
                    title="Xu Hướng Giá Trung Bình Theo Nền Tảng")
        plt.ylabel('Giá Trung Bình (VNĐ)')
        plt.xlabel('Thời Gian')
        plt.legend(title='Nền Tảng', loc='best')
        plt.tight_layout()
        plt.savefig('price_trends.png', dpi=300)
        plt.close()
        
    except Exception as e:
        print(f"Lỗi khi tạo biểu đồ: {e}")
        raise

def generate_html_report(df):
    """Tạo báo cáo HTML chi tiết và chuyên nghiệp"""
    # Tính toán các chỉ số quan trọng
    total_products = len(df)
    platform_breakdown = df['platform'].value_counts(normalize=True) * 100
    avg_rating = df['rating'].mean()
    median_rating = df['rating'].median()
    avg_price = df['salePrice'].mean()
    median_price = df['salePrice'].median()
    price_range = df['salePrice'].max() - df['salePrice'].min()

    # Phân tích chi tiết
    top_platforms = platform_breakdown.head(3)
    rating_distribution = df['rating'].value_counts(normalize=True).sort_index() * 100

    html_content = f"""
    <html>
        <body style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; 
                     max-width: 800px; 
                     margin: 0 auto; 
                     padding: 20px; 
                     line-height: 1.6; 
                     color: #333;">
            <div style="background-color: #f0f4f8; 
                        padding: 20px; 
                        border-radius: 10px; 
                        text-align: center; 
                        margin-bottom: 20px;">
                <h1 style="color: #1a5f7a; 
                           border-bottom: 3px solid #1a5f7a; 
                           padding-bottom: 10px;">
                    BÁO CÁO PHÂN TÍCH THƯƠNG MẠI ĐIỆN TỬ
                </h1>
                <p style="font-style: italic; color: #666;">
                    Báo cáo chi tiết về xu hướng và hiệu suất sản phẩm
                </p>
            </div>

            <div style="background-color: #e9f5f9; 
                        padding: 15px; 
                        border-radius: 8px; 
                        margin-bottom: 20px;">
                <h2 style="color: #2c3e50;">Tổng Quan Báo Cáo</h2>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr>
                        <td><strong>Thời Gian Báo Cáo:</strong></td>
                        <td>{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}</td>
                    </tr>
                    <tr>
                        <td><strong>Tổng Sản Phẩm:</strong></td>
                        <td>{total_products:,}</td>
                    </tr>
                    <tr>
                        <td><strong>Điểm Đánh Giá Trung Bình:</strong></td>
                        <td>{avg_rating:.2f}/5.00 (Trung vị: {median_rating:.2f})</td>
                    </tr>
                    <tr>
                        <td><strong>Giá Sản Phẩm Trung Bình:</strong></td>
                        <td>{avg_price:,.0f} VNĐ (Trung vị: {median_price:,.0f} VNĐ)</td>
                    </tr>
                </table>
            </div>

            <div style="margin-bottom: 20px;">
                <h2 style="color: #2c3e50;">Phân Tích Chi Tiết</h2>
                
                <h3>Phân Bố Nền Tảng</h3>
                <p>Top 3 nền tảng hàng đầu:</p>
                {''.join([f'<p>• <strong>{platform}</strong>: {percentage:.2f}%</p>' for platform, percentage in top_platforms.items()])}
                <img src="cid:platform_dist" alt="Phân Bố Nền Tảng" style="max-width: 100%; height: auto;">
                
                <h3>Phân Bố Điểm Đánh Giá</h3>
                <p>Chi tiết điểm đánh giá của sản phẩm:</p>
                {''.join([f'<p>• <strong>{rating} Sao</strong>: {percentage:.2f}%</p>' for rating, percentage in rating_distribution.items()])}
                <img src="cid:rating_dist" alt="Phân Bố Điểm Đánh Giá" style="max-width: 100%; height: auto;">
                
                <h3>Phân Phối Giá</h3>
                <p>Phân tích chi tiết về giá sản phẩm:</p>
                <p>• Khoảng giá: {price_range:,.0f} VNĐ</p>
                <img src="cid:price_dist" alt="Phân Phối Giá" style="max-width: 100%; height: auto;">
                
                <h3>Xu Hướng Giá Theo Thời Gian</h3>
                <p>Theo dõi sự biến động giá trung bình trên các nền tảng:</p>
                <img src="cid:price_trends" alt="Xu Hướng Giá" style="max-width: 100%; height: auto;">
            </div>

            <div style="background-color: #f0f4f8; 
                        padding: 15px; 
                        border-radius: 8px; 
                        text-align: center;">
                <p style="color: #666; font-style: italic;">
                    Báo cáo được tạo tự động - Để biết thêm chi tiết, vui lòng liên hệ bộ phận phân tích
                </p>
            </div>
        </body>
    </html>
    """
    return html_content

def send_email(sender, app_password, recipient):
    """Gửi email báo cáo với các thông tin chi tiết"""
    # Kết nối cơ sở dữ liệu
    con = sqlite3.connect('e-com.sqlite')
    df = pd.read_sql_query("SELECT * FROM fact_sales;", con)
    con.close()

    # Chuyển đổi timestamp
    df['scraped_timestamp'] = pd.to_datetime(df['scraped_timestamp'])

    # Tạo biểu đồ
    generate_and_save_plots(df)

    # Tạo thông điệp email
    msg = MIMEMultipart('related')
    msg['Subject'] = f'Báo Cáo Phân Tích Thương Mại Điện Tử - {datetime.now().strftime("%d/%m/%Y")}'
    msg['From'] = sender
    msg['To'] = recipient

    # Tạo nội dung HTML
    html_content = generate_html_report(df)
    html_part = MIMEText(html_content, 'html', 'utf-8')
    msg.attach(html_part)

    # Đính kèm hình ảnh
    images = {
        'platform_dist.png': 'platform_dist',
        'rating_dist.png': 'rating_dist',
        'price_dist.png': 'price_dist',
        'price_trends.png': 'price_trends'
    }
    
    for image_path, cid in images.items():
        with open(image_path, 'rb') as img_file:
            img = MIMEBase('application', 'octet-stream')
            img.set_payload(img_file.read())
            encoders.encode_base64(img)
            img.add_header('Content-Disposition', f'inline; filename={image_path}')
            img.add_header('Content-ID', f'<{cid}>')
            msg.attach(img)

    try:
        # Kết nối máy chủ email an toàn
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.login(sender, app_password)
        
        # Gửi email
        server.sendmail(sender, recipient, msg.as_string())
        server.quit()
        print("Gửi email thành công!")
        
    except Exception as e:
        print(f"Lỗi khi gửi email: {str(e)}")

if __name__ == '__main__':
    # Cấu hình email
    sender = "~~~~~~~~~~~~@gmail.com"
    app_password = "~~~~~~~~~~~"
    recipient = "~~~~~~~~~~~~~~~~@gmail.com"
    
    # Gửi báo cáo
    send_email(sender, app_password, recipient)

Gửi email thành công!


<Figure size 1200x800 with 0 Axes>